In [16]:
# Initial imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statistics import mean, mode
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV, cross_val_score, cross_validate
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
# Loading data
file_path = Path("../data/myopia.csv")
df = pd.read_csv(file_path)
df.head(5)

,AGE,SPHEQ,AL,ACD,LT,VCD,SPORTHR,READHR,COMPHR,STUDYHR,TVHR,DIOPTERHR,MOMMY,DADMY,MYOPIC
0,6,-0.052,21.889999,3.690,3.498,14.70,45,8,0,0,10,34,1,1,1
1,6,0.608,22.379999,3.702,3.392,15.29,4,0,1,1,7,12,1,1,0
2,6,1.179,22.490000,3.462,3.514,15.52,14,0,2,0,10,14,0,0,0
3,6,0.525,22.200001,3.862,3.612,14.73,18,11,0,0,4,37,0,1,1
4,5,0.697,23.290001,3.676,3.454,16.16,14,0,0,0,4,4,1,0,0


In [3]:
# Check dataset balance
df["MYOPIC"].value_counts()

0    537
1     81
Name: MYOPIC, dtype: int64

In [4]:
# Check dataset balance
df["MYOPIC"].value_counts(normalize=True)

0    0.868932
1    0.131068
Name: MYOPIC, dtype: float64

In [5]:
# Define X,y
label = df["MYOPIC"]
X = df.iloc[:,:-1].copy()
X.head()

,AGE,SPHEQ,AL,ACD,LT,VCD,SPORTHR,READHR,COMPHR,STUDYHR,TVHR,DIOPTERHR,MOMMY,DADMY
0,6,-0.052,21.889999,3.690,3.498,14.70,45,8,0,0,10,34,1,1
1,6,0.608,22.379999,3.702,3.392,15.29,4,0,1,1,7,12,1,1
2,6,1.179,22.490000,3.462,3.514,15.52,14,0,2,0,10,14,0,0
3,6,0.525,22.200001,3.862,3.612,14.73,18,11,0,0,4,37,0,1
4,5,0.697,23.290001,3.676,3.454,16.16,14,0,0,0,4,4,1,0


## Pipeline with SVC and Cross-Validation

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, label, random_state=42)

In [7]:
# create process steps
pipes = [
    ("scaler", StandardScaler()),
    ("pca", PCA()),
    ("svc", SVC())
]

In [8]:
# create pipeline
pipeline = Pipeline(pipes)

In [9]:
# set gridsearch parameters
params = {
    "svc__C": [0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 50, 100, 1000],
    "svc__gamma": [0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 100],
    "pca__n_components":[10]
}

In [10]:
# gridsearch setup
grid = GridSearchCV(pipeline, params)

In [11]:
# Train the scaler with the X_train data.
grid.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()), ('svc', SVC())]),
             param_grid={'pca__n_components': [10],
                         'svc__C': [0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 50, 100,
                                    1000],
                         'svc__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 5, 10,
                                        100]})

In [12]:
# evaluate model
grid.score(X_train, y_train)

0.9028077753779697

In [13]:
y_train_pred = grid.predict(X_train)

In [14]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95       400
           1       0.82      0.37      0.51        63

    accuracy                           0.90       463
   macro avg       0.86      0.68      0.73       463
weighted avg       0.90      0.90      0.89       463



In [17]:
scoring = ('f1', 'recall', 'precision', 'roc_auc')

# evaluate pipeline
# Note for imbalanced classification don't use k-fold cross-validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_validate(pipeline, X_train, y_train, scoring=scoring, cv=cv, n_jobs=-1)
print('Mean f1: %.3f' % mean(scores['test_f1']))
print('Mean recall: %.3f' % mean(scores['test_recall']))
print('Mean precision: %.3f' % mean(scores['test_precision']))
print('Mean ROC AUC: %.3f' % mean(scores['test_roc_auc']))
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(f"-----"*10)
print(classification_report(y_test, y_pred))

Mean f1: 0.062
Mean recall: 0.037
Mean precision: 0.217
Mean ROC AUC: 0.811
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.88      1.00      0.94       137
           1       0.00      0.00      0.00        18

    accuracy                           0.88       155
   macro avg       0.44      0.50      0.47       155
weighted avg       0.78      0.88      0.83       155



C:\Users\ejwda\Anaconda3\envs\PythonDataS2022\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ejwda\Anaconda3\envs\PythonDataS2022\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ejwda\Anaconda3\envs\PythonDataS2022\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [18]:
# gridsearch results
grid.best_params_

{'pca__n_components': 10, 'svc__C': 1000, 'svc__gamma': 0.001}

In [19]:
# make predictions
y_pred = grid.predict(X_test)
accuracy_score(y_test, y_pred)

0.8903225806451613

In [20]:
confusion_matrix(y_test, y_pred)

array([[134,   3],
       [ 14,   4]], dtype=int64)

In [21]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94       137
           1       0.57      0.22      0.32        18

    accuracy                           0.89       155
   macro avg       0.74      0.60      0.63       155
weighted avg       0.87      0.89      0.87       155



## SVC with Balanced Data

In [22]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [23]:
ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(X_train, y_train)

In [24]:
print('Original dataset shape', Counter(y_train))
print('Resample dataset shape', Counter(y_ros))

Original dataset shape Counter({0: 400, 1: 63})
Resample dataset shape Counter({0: 400, 1: 400})


In [25]:
# create pipeline
pipeline_bal = Pipeline(pipes)

In [26]:
# set gridsearch parameters
params_bal = {
    "svc__C": [0.0001, 0.001, 0.01, 0.05, 0.1, 0.18],
    "svc__gamma": [0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 100],
    "pca__n_components":[10]
}

In [27]:
# gridsearch setup
grid_bal = GridSearchCV(pipeline_bal, params_bal)

In [28]:
# Train the scaler with the balanced training data
grid_bal.fit(x_ros, y_ros)

GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()), ('svc', SVC())]),
             param_grid={'pca__n_components': [10],
                         'svc__C': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.18],
                         'svc__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 5, 10,
                                        100]})

In [29]:
# make predictions
y_pred = grid_bal.predict(X_train)
accuracy_score(y_train, y_pred)

0.8336933045356372

In [30]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.82      0.90       400
           1       0.45      0.90      0.60        63

    accuracy                           0.83       463
   macro avg       0.71      0.86      0.75       463
weighted avg       0.91      0.83      0.85       463



In [36]:
scoring = ('f1', 'recall', 'precision', 'roc_auc')

# evaluate pipeline
# Note for imbalanced classification don't use k-fold cross-validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_validate(pipeline_bal, X_train, y_train, scoring=scoring, cv=cv, n_jobs=-1)
print('Mean f1: %.3f' % mean(scores['test_f1']))
print('Mean recall: %.3f' % mean(scores['test_recall']))
print('Mean precision: %.3f' % mean(scores['test_precision']))
print('Mean ROC AUC: %.3f' % mean(scores['test_roc_auc']))
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(f"-----"*10)
print(classification_report(y_test, y_pred))

Mean f1: 0.062
Mean recall: 0.037
Mean precision: 0.217
Mean ROC AUC: 0.811
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.88      1.00      0.94       137
           1       0.00      0.00      0.00        18

    accuracy                           0.88       155
   macro avg       0.44      0.50      0.47       155
weighted avg       0.78      0.88      0.83       155



C:\Users\ejwda\Anaconda3\envs\PythonDataS2022\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ejwda\Anaconda3\envs\PythonDataS2022\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ejwda\Anaconda3\envs\PythonDataS2022\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [32]:
# gridsearch results
grid_bal.best_params_

{'pca__n_components': 10, 'svc__C': 0.18, 'svc__gamma': 0.1}

In [33]:
# make predictions
y_pred = grid_bal.predict(X_test)
accuracy_score(y_test, y_pred)

0.7806451612903226

In [34]:
confusion_matrix(y_test, y_pred)

array([[112,  25],
       [  9,   9]], dtype=int64)

In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.82      0.87       137
           1       0.26      0.50      0.35        18

    accuracy                           0.78       155
   macro avg       0.60      0.66      0.61       155
weighted avg       0.85      0.78      0.81       155



## Analysis


I think SVC with the balanced data training set is over-fitting.  It's predicing perfectly in training 
but not predicting the minority outcome.  

Need to forloop changing C until f1 is optimized.